In [ ]:
##########################################################################################################
        labelmap2vol = npl.inv(target_affine).dot(labelmap_affine)
        seg_start_inv = apply_affine(labelmap2vol, [0,0,0]).astype(np.int32)
        seg_end_inv = apply_affine(labelmap2vol, segm_dim_v).astype(np.int32)
        print('look:', labelmap2vol, labelmap.shape)
#         s = labelmap.shape
#         labelmap_r = labelmap.reshape(-1, 3)
        
#         print(labelmap_r.shape)
#         seg_outs = apply_affine(labelmap2vol, labelmap_r)
#         labelmap = seg_outs.reshape(s)
        
        aff = img_reference_r.affine
        print('orig affine:', aff)
        aff[:3, 3] = q_offsets_l
        print('copied affine:', aff)
        labelmap_x = resample_from_to(labelmap_img, [img_reference_r.shape, aff])
        
#         labelmap = resample_img(
#         labelmap_img,
#         target_affine=target_affine,
#         target_shape=(np.array(im_dim_v)).astype(int))

        print(labelmap_x.shape)



        empty_header = nb.Nifti1Header()
        stitched_labeled_img = nb.Nifti1Image(labelmap_x, target_affine, empty_header)
        save_volume(labelmap_x, f'nako_label_{lidx}')
        

In [ ]:
 #---------------------------------------------------------------------------------------------------
        print("seg start inv v: ",seg_start_inv , "segm end inv v:",seg_end_inv)
        
        shifts = seg_start_inv #tuple(labelmap2vol[:3, 3].astype(np.int32))
        print("segm start v with inv::", shifts)
    #     print(source_data.shape)
        for ax, shift in enumerate(shifts):
            print(ax, shift)
            shift = int(shift)
            if shift < 0:
                labelmap = flip_axis(labelmap, axis=ax)
#             print(-np.abs(shift))
#             source_data = np.roll(source_data, -np.abs(shift), axis=ax)
        segm_start_x, segm_start_y, segm_start_z = np.absolute(shifts)

In [ ]:
volume = img_ras_sampled_combined
print("STARTING NAKO LABEL-MAPS.")
print('Reading Label Maps.....')
label_files_to_read_list = glob.glob('./datasets/nako/100017_label/**')
print(label_files_to_read_list)

img_ras_list = []
img_ras_sampled_combined_labelmap = None
header, affine = None, None
for lidx, label_file_to_read in enumerate(label_files_to_read_list):
    img_ras = read_ras(label_file_to_read)
    mode='nearest'
    print(img_ras.header)
    
    target_affine = volume.affine.copy()
#     target_affine[:, 3] = img_ras.affine[:, 3].copy()
    print(volume.affine, target_affine)
    img_ras = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    img_ras = placing_axes(img_ras, volume.affine.copy(), volume.header.copy())
    img_ras_sampled = resample_from_to(img_ras, [volume.shape, target_affine])
    
#     img_ras_sampled = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
#     print(img_ras_sampled.header)

    labelmap = img_ras_sampled.get_fdata()
    labelmap = np.multiply(lidx, labelmap)
    if header is None or affine is None:
        header = img_ras_sampled.header
        affine = img_ras_sampled.affine
        
    if img_ras_sampled_combined_labelmap is None:
        img_ras_sampled_combined_labelmap = labelmap
    else:
        img_ras_sampled_combined_labelmap = np.add(img_ras_sampled_combined_labelmap, labelmap)
        
img_ras_sampled_combined_labelmap = nb.Nifti1Image(img_ras_sampled_combined_labelmap, affine, header)

print('Merging/Stitching Label-Maps.....')
label = placing_axes(img_ras_sampled_combined_labelmap, volume.affine, volume.header)

print('Viewing Stitched Images.....')

volume_3_view_viewer(get_volume_data(img_ras_sampled_combined_labelmap))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined_labelmap, 'nako_ras_sampled_stitched_labelmap')
print('FINISHED.')

In [ ]:
print("STARTING KORA.")
print('Reading Volumes.....')
file_to_read1 = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz'
file_to_read2 = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz'

img1_ras = read_ras(file_to_read1)
img2_ras = read_ras(file_to_read2)

print('Merging/Stitching Images.....')
img_ras_sampled_combined = multi_vol_stitching([img1_ras, img2_ras])

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(img_ras_sampled_combined))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined, 'kora_ras_sampled_stitched')
print('FINISHED.')

In [ ]:
# file_structure = [[[v1,v2],[l1,l2,l3,l4,l5,l6,l7,l8],[...]]
print("STARTING KORA LABEL-MAPS.")
print('Reading Label Maps.....')
label_files_to_read_list = glob.glob('./datasets/kora/KORA2460249_label/**')
print(label_files_to_read_list, img_ras_sampled_combined.header)

img_ras_list = []
img_ras_sampled_combined_labelmap = None
header, affine = None, None
for lidx, label_file_to_read in enumerate(label_files_to_read_list):
    img_ras = read_ras(label_file_to_read)
    mode='nearest'
    print(img_ras.header)
    
    target_affine = img_ras_sampled_combined.affine.copy()
    target_affine[:, 3] = img_ras.affine[:, 3].copy()
    print(img_ras_sampled_combined.affine, target_affine)
    img_ras_sampled = resample_from_to(img_ras, [img_ras_sampled_combined.shape, target_affine])
    
#     img_ras_sampled = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    print(img_ras_sampled.header)

    labelmap = img_ras_sampled.get_fdata()
    labelmap = np.multiply(lidx, labelmap)
    if header is None or affine is None:
        header = img_ras_sampled.header
        affine = img_ras_sampled.affine
        
    if img_ras_sampled_combined_labelmap is None:
        img_ras_sampled_combined_labelmap = labelmap
    else:
        img_ras_sampled_combined_labelmap = np.add(img_ras_sampled_combined_labelmap, labelmap)
        
img_ras_sampled_combined_labelmap = nb.Nifti1Image(img_ras_sampled_combined_labelmap, affine, header)
#     img_ras_list.append(img_ras_sampled)

print('Merging/Stitching Label-Maps.....')
# img_ras_sampled_combined_labelmap = multi_vol_stitching(img_ras, True)
# target_affine = img_0.affine.copy()
# target_affine[2,3] = img_1.affine[2,3].copy()
# target_shape = img_0.shape[:2] + img_1.shape[2:]
img_ras_sampled_combined_labelmap_vol = np.flip(np.moveaxis(get_volume_data(img_ras_sampled_combined_labelmap), 2, 1))
img_ras_sampled_combined_labelmap = nb.Nifti1Image(img_ras_sampled_combined_labelmap_vol, img_ras_sampled_combined_labelmap.affine, img_ras_sampled_combined_labelmap.header)
img_ras_sampled_combined_labelmap = resample_from_to(img_ras_sampled_combined_labelmap, [img_ras_sampled_combined.shape, img_ras_sampled_combined.affine.copy()])
# img_ras_sampled_combined_labelmap = nb.funcs.concat_images(img_ras_list, axis=2)
# label = placing_axes(img_ras_sampled_combined_labelmap, img_ras_sampled_combined)

print('Viewing Stitched Images.....')

volume_3_view_viewer(get_volume_data(img_ras_sampled_combined_labelmap))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined_labelmap, 'kora_ras_sampled_stitched_labelmap')
print('FINISHED.')

In [ ]:
print('Viewing Stitched Images.....')
img_ras_sampled_combined_labelmap = np.flip(np.moveaxis(img_ras_sampled_combined_labelmap, 2, 1)) 
volume_3_view_viewer(get_volume_data(img_ras_sampled_combined_labelmap))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined_labelmap, 'kora_ras_sampled_stitched_labelmap')
print('FINISHED.')

In [ ]:
vol = img_ras_sampled_combined
label = img_ras_sampled_combined_labelmap

label = resample_from_to(label, [vol.shape, vol.affine])
print(vol.header, label.header)
# label = multi_vol_stitching([vol, label])

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(vol))

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(label))

save_volume(vol, 'kora_final_vol')
save_volume(label, 'kora_final_label')

In [ ]:
print("STARTING UKB.")
print('Reading Volumes.....')
file_to_read1 = './datasets/ukb/5331775_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
file_to_read2 = './datasets/ukb/5331775_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz'
file_to_read3 = './datasets/ukb/5331775_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'

img1_ras = read_ras(file_to_read1)
img2_ras = read_ras(file_to_read2)
img3_ras = read_ras(file_to_read3)

print('Merging/Stitching Images.....')
img_ras_sampled_combined = multi_vol_stitching([img1_ras, img2_ras, img3_ras])

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(img_ras_sampled_combined))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined, 'ukb_ras_sampled_stitched')
print('FINISHED.')

In [ ]:
for ax, shift in enumerate(seg_start_inv):
            print(ax, shift)
            shift = int(shift)
            if ax !=2:
                if shift < 0:
                    labelmap = flip_axis(labelmap, axis=ax)
                    seg_end_inv[ax] += abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
                else:
                    seg_end_inv[ax] -= abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0

In [1]:
# IMPORTANT TESTINGSS HERE...

In [ ]:
def do_nibabel_transform_to_ras(img):
    print(f'Transforming Images to {DEFAULT_ORIENTATION}.....')
    affine = img.affine
    orig_ornt = nb.io_orientation(affine)
    targ_ornt = axcodes2ornt(DEFAULT_ORIENTATION)
    transform = ornt_transform(orig_ornt, targ_ornt)
    img = img.as_reoriented(transform)
    return img

def multi_vol_stitching(images, is_label=False):
    if len(images)==1:
        return images
    elif len(images) ==0:
        raise Exception("Empty Image List!")
        
    images_sorted = sorted(images, key=lambda im: im.header['qoffset_z'], reverse=True)
    img_0 = images_sorted[0]
    
    mode = 'nearest' if is_label else 'constant'
    img_0 = resample_to_output(img_0, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    
    for idx, img_1 in enumerate(images_sorted[1:]):
        print(f'{idx}th img for stitching...')
#         print("STARTTTTT: ", img_1.affine)
        img_1 = resample_to_output(img_1, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
#         img_1 = placing_axes(img_1, img_0.affine.copy(), img_0.header.copy(), [2])
#         print( img_1.affine)
        target_affine = img_0.affine.copy()
        target_affine[2,3] = img_1.affine[2,3].copy()
        target_shape = img_0.shape[:2] + img_1.shape[2:]
#         img_1 = placing_axes(img_1, target_affine, img_0.header.copy(), skip_axis=[2])
        img_1 = resample_from_to(img_1, [target_shape, target_affine])
#         print(img_1.affine)
        img_0 = vol_stitching(img_0, img_1)
#         print(img_0.affine, 'FINISHHHHHHHHH')
        
    return img_0
    
def vol_stitching(im_0, im_1):
    im_0_z = im_0.shape[2]
    im_1_z = im_1.shape[2]

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']

    spacing = im_0.header['pixdim'][3]

    im_0_width = im_0_z * spacing
    im_1_width = im_1_z * spacing

    im_1_start = im_1_end + im_1_width
    im_0_start = im_0_end + im_0_width

    overlap = abs(im_0_end - im_1_start)

    overlap_v = int(round(overlap / spacing))

    new_im_dim = abs(round((abs(im_1_end - im_0_start)) / spacing))

    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim)])

    im_0_data = im_0.get_fdata()
    im_1_data = im_1.get_fdata()

    new_img[:, :, 0:(im_1_z - overlap_v)] = im_1_data[:, :, 0:(im_1_z - overlap_v)]
    new_img[:, :, im_1_z:] = im_0_data[:, :, overlap_v:]
    
    # overlap region:
    sigmoid_c = sigmoid(np.linspace(-DEFAULT_LINSPACE, DEFAULT_LINSPACE, overlap_v))
    
    for l in range(0, overlap_v):
        new_img[:, :, (im_1_z - overlap_v + l)] = \
        (1 - sigmoid_c[l]) * im_1_data[:,:, (im_1_z - overlap_v) + l] + (sigmoid_c[l]) * im_0_data[:, :, l]
        
    stitched_img = nb.Nifti1Image(new_img, im_1.affine, im_1.header)
#     placing_axes(stitched_img)
    return stitched_img

def read_ras(file_path, file_type=None, is_label=False):
    _, _, img = file_reader(file_path, file_type)
    img_ras = do_nibabel_transform_to_ras(img)
    if is_label:
        file_labels = list(FILE_TO_LABEL_MAP.keys())
        lidx, labelname = fetch_class_labels_from_filemap(file_path, file_labels)
        return img_ras, lidx, labelname
    else:
        return img_ras

def fetch_class_labels_from_filemap(labelmap_path, file_labels):
    label_idx, label = None, None
    for lidx, file_label in enumerate(file_labels):
        if file_label in labelmap_path.replace(" ", "").upper():
            label_idx, label = lidx, file_label
            break
    return label_idx, label


def placing_axes(vol, target_affine, target_header=None, skip_axis=None):
    
    vol2target = npl.inv(target_affine).dot(vol.affine)
    source_data = vol.get_fdata()
    shifts = tuple(vol2target[:3, 3].astype(np.int32))
    print(shifts)
#     print(source_data.shape)
    for ax, shift in enumerate(shifts):
        if skip_axis is not None and ax in skip_axis:
            continue
        print(ax, shift)
        shift = int(shift)
        if shift < 0:
            source_data = flip_axis(source_data, axis=ax)
        print(-np.abs(shift))
        source_data = np.roll(source_data, -np.abs(shift), axis=ax)
                
    if target_header is None:
        target_header = nb.Nifti1Header()
    stitched_labeled_img = nb.Nifti1Image(source_data, target_affine, target_header)

    return stitched_labeled_img


def label_stitching(label_parts, img_reference):
    img_reference_r = img_reference
    processed_segm = None
    header = img_reference.header
    target_affine = img_reference.affine
    steps = header['pixdim'][1:4]
    v_vol = np.prod(img_reference.shape)
    volume = img_reference.get_fdata()
    im_dim_v = volume.shape
    
    processed_segm = np.zeros_like(volume)

#     q_offsets = list(header.get_best_affine()[:3, 3]) 
    im_offx = abs(header['qoffset_x'])
    im_offy = abs(header['qoffset_y'])
    im_offz = abs(header['qoffset_z'])
    q_offsets = [im_offx,im_offy,im_offz]
    
    reference_labelmap = None
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)
        if lname is not'SPLEEN' and lname is not'LIVER':
            continue
            
        if reference_labelmap is None:
            reference_labelmap = labelmap_img
        else:
            labelmap_img = resample_from_to(labelmap_img, [reference_labelmap.shape, reference_labelmap.affine])
            print('c shape:', labelmap_img.shape, labelmap_img.affine)
            
#         labelmap_img.set_data_dtype(np.uint16)
            
        label_header = labelmap_img.header
        labelmap = labelmap_img.get_fdata()
        labelmap_affine = labelmap_img.affine
        steps_l = label_header['pixdim'][1:4]
#         q_offsets_l = list(label_header.get_best_affine()[:3, 3])
        lblmp_offx = abs(label_header['qoffset_x'])
        lblmp_offy = abs(label_header['qoffset_y'])
        lblmp_offz = abs(label_header['qoffset_z'])
        q_offsets_l = [lblmp_offx,lblmp_offy,lblmp_offz]
        print(f'vol offsets: {q_offsets}')
        print(f'label_offsets: {q_offsets_l}')
        l_vol = np.prod(labelmap.shape)
        segm_dim_v = labelmap.shape
        
        print(label_header)
        
        print(f'vol shape: {im_dim_v}, label shape: {segm_dim_v}')
        
        # shape to width
        vol_dim_w = im_dim_v * steps
        segm_dim_w = segm_dim_v * steps_l
        
        print(f'vol width: {vol_dim_w}, label width: {segm_dim_w}')
        
        vol_start = q_offsets
        label_start = q_offsets_l
        
        vol_end = np.array([vol_start[0]+vol_dim_w[0], vol_start[1]+vol_dim_w[1], vol_start[2]+vol_dim_w[2]])
        label_end = np.array([label_start[0]+segm_dim_w[0], label_start[1]+segm_dim_w[1], label_start[2]+segm_dim_w[2]])
        
        print(f'vol start: {vol_start}, label start: {label_start}')
        print(f'vol end: {vol_end}, label end: {label_end}')
        
        end_diff_w = vol_end - label_end
        
        print('end_diff w: ', end_diff_w)
        end_diff_v = np.round(end_diff_w / steps_l).astype(int)
        print('end diff v: ', end_diff_v)
        
        # Difference sign does not matter
        segm_end_v = im_dim_v - end_diff_v
        
        segm_end_x = segm_end_v[0]
        segm_end_y = segm_end_v[1]
        segm_end_z = segm_end_v[2]
        
        segm_start_x = segm_end_v[0] - segm_dim_v[0]
        segm_start_y = segm_end_v[1] - segm_dim_v[1]
        segm_start_z = segm_end_v[2] - segm_dim_v[2]
        
        segm_end_x,segm_end_y,segm_end_z = segm_start_x+segm_dim_v[0],segm_start_y+segm_dim_v[1],segm_start_z+segm_dim_v[2]
                
        labelmap = np.multiply(lidx, labelmap)
        print('current labelmap shape:', labelmap.shape, 'processed segm shape:', processed_segm.shape)
        
        print('process seg slice shape:',processed_segm[0:segm_end_x, 0:segm_end_y, segm_start_z:segm_end_z].shape)
#         print(segm_start_x,segm_end_x, segm_start_y,segm_end_y, segm_start_z,segm_end_z)
        print("segm start v:", [segm_start_x, segm_start_y, segm_start_z], 'segm end v: ', [segm_end_x,segm_end_y,segm_end_z])
    
        ############
        labelmap2vol = npl.inv(target_affine).dot(labelmap_affine)
        seg_start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
        seg_end_inv = apply_affine(labelmap2vol, segm_dim_v).astype(np.int32)
        print("seg start inv v: ",seg_start_inv , "segm end inv v:",seg_end_inv)
        ############
        shifts = tuple(np.floor(labelmap2vol[:3, 3]).astype(np.int32))
        for ax, shift in enumerate(seg_start_inv):
            print(ax, shift)
            shift = int(shift)
            if ax !=2:
                labelmap = flip_axis(labelmap, axis=ax)
                if shift < 0:
                    seg_end_inv[ax] += abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
                else:
                    seg_end_inv[ax] -= abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0

#         if lname is 'SPLEEN':
#             labelmap = flip_axis(labelmap, axis=0)
#             labelmap = flip_axis(labelmap, axis=1)
#             seg_start_inv = [51, 35, 209]
#             seg_end_inv = [90, 70, 237]
#             seg_end_inv[2] += abs(seg_start_inv[2])
#             seg_start_inv[2] = 0
            
#             seg_end_inv[1] += abs(seg_start_inv[1])
#             seg_start_inv[1] = 0
            
#         neg_count = len(list(filter(lambda x: (x < 0), np.sign(q_offsets + q_offsets_l))))
    
        print("updated seg start inv v: ",seg_start_inv , "updated segm end inv v:",seg_end_inv)
        try:
#             processed_segm[0:segm_end_x, 0:segm_end_y, segm_start_z:segm_end_z] += labelmap
            processed_segm[seg_start_inv[0]:seg_end_inv[0], seg_start_inv[1]:seg_end_inv[1], seg_start_inv[2]:seg_end_inv[2]] += labelmap
        except Exception as e:
            print("ERRORRRRR:", e)
#             labelmap = flip_axis(labelmap, axis=0)
#             labelmap = flip_axis(labelmap, axis=1)
            processed_segm[segm_start_x+8:segm_end_x, segm_start_y+3:segm_end_y, segm_start_z:segm_end_z] += labelmap
        finally:
            print("###############################################################################################")
        
        
    labelmap = processed_segm
    labelmap = np.round(labelmap)
    volume = normalise_data(volume)

    empty_header = nb.Nifti1Header()
    stitched_labeled_img = nb.Nifti1Image(labelmap, target_affine, empty_header)
    volume_img = nb.Nifti1Image(volume, target_affine, empty_header)
    
    return volume_img, stitched_labeled_img
    